In [18]:
from dash import dcc, html, Output, Input
from jupyter_dash import JupyterDash
import plotly.express as px
import base64, pandas as pd
from datetime import date

In [2]:
# specify the directory of the project
proj_dir = '../'

# read the data that matches the specified DateTime to the location of the image
df = pd.read_csv(f'{proj_dir}data/bkk_all.csv').drop('Unnamed: 0', axis=1)
df.head()

,DateTime,FolderPath,FileName
0,2020-06-18 06:30:00,/data/bkk_radar_images/bkk_radar_images_1/,1592461954.png
1,2020-06-18 06:35:00,/data/bkk_radar_images/bkk_radar_images_1/,1592462136.png
2,2020-06-18 06:45:00,/data/bkk_radar_images/bkk_radar_images_1/,1592462768.png
3,2020-06-18 06:50:00,/data/bkk_radar_images/bkk_radar_images_1/,1592463068.png
4,2020-06-18 06:55:00,/data/bkk_radar_images/bkk_radar_images_1/,1592463277.png


In [10]:
# extract the DateTime column
DateTime = df['DateTime']
DateTime.head(3)

0    2020-06-18 06:30:00
1    2020-06-18 06:35:00
2    2020-06-18 06:45:00
Name: DateTime, dtype: object

In [15]:
# extract the Date
Date = DateTime.str.split(' ').str[0].unique()

min_date_allowed = [int(i) for i in Date[0].split('-')]
max_date_allowed = [int(j) for j in Date[-1].split('-')]

In [16]:
# extract the timestamp and create a dictionary
Time = {}
for d in Date:
    Time[d] = DateTime[DateTime.str.contains(d)].str.split(' ').str[1]

In [19]:
# Dash section
app = JupyterDash(__name__)

app.layout = html.Div([

    html.Div([
        html.Div([
            html.H1('Nowcasting', className='text-white'),

            html.Div([
                html.Div([

                    html.P('Date: ', className='text-white'),
                    dcc.DatePickerSingle(
                        id='date-picker',
                        min_date_allowed=date(min_date_allowed[0], min_date_allowed[1], min_date_allowed[2]),
                        max_date_allowed=date(max_date_allowed[0], max_date_allowed[1], max_date_allowed[2]),
                        initial_visible_month=date(min_date_allowed[0], min_date_allowed[1], min_date_allowed[2]),
                        date=date(min_date_allowed[0], min_date_allowed[1], min_date_allowed[2]),
                    ),
                ]),
                html.Div([    
                    html.P('Time: ', className='text-white'),
                    dcc.Dropdown(id='time-dropdown', className='dropdown-list')
                ])
            ], style={'display': 'flex', 'gap': '10px'}),

            ], 
        style={'display': 'flex', 'flexDirection': 'column'}),

        html.Div([
            html.Img(id='image', className='img-width-height bg-white')
            ]),
        html.Div([
            html.Img(id='image-2', className='img-width-height bg-white')
            ]),
        html.Div([
            html.Img(id='image-3', className='img-width-height bg-white')
            ]),

    ], className='container'),
])

@app.callback(
    Output('time-dropdown', 'options'),
    Output('time-dropdown', 'value'),
    Input('date-picker', 'date')
)
def updateTimeDropdown(date):
    date = str(date)
    return ([{'label': i, 'value': i} for i in Time[date]], Time[date].iloc[0])

@app.callback(
    Output('image', 'src'),
    Output('image-2', 'src'),
    Output('image-3', 'src'),
    Input('date-picker', 'date'),
    Input('time-dropdown', 'value'),
)
def updateImg(date, time):
    result = df.loc[df['DateTime'] == f'{date} {time}', ['FolderPath', 'FileName']]
    path = '../' + result.iloc[0,0] + result.iloc[0,1]
    encoded_image = base64.b64encode(open(path, 'rb').read())
    ret = 'data:image/png;base64,{}'.format(encoded_image.decode())
    return (ret, ret, ret)

if __name__ == '__main__':
    app.run_server(mode='inline')